In [4]:
import requests #Connect to endpoints
import time #time script
import sys
import csv
import pandas as pd
import spotipy #authentication
import spotipy.util as util #authentication

from spotipy.oauth2 import SpotifyClientCredentials #authentication

In [5]:
#Credentials
cid = 'f61689bf814a44dab11d7e63d18dc38a'
secret = '7d2c54296b204a20b26b1c7eb5a93601'
redirect_uri = 'http://google.com/'
username = '31tabw4566sgjjakrw5afmpspdv4'

In [6]:
#Authentication
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=f61689bf814a44dab11d7e63d18dc38a&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: https://www.google.com/?code=AQC87xV2_IcEa-GDGzK5WtnVkQx-aD2gE2xfQLSMLCGeyA2YcqAEjo8u1ore5gja9t49DeIB4PnfqrHYB125r7E7lEmWkMvWCp9aeawU9iJ7uspTpFBGUQn_rA3V2xuG08AfkWS7ZL6tKUUjNJZSmppLGIwV0h5__WVrMn7wdZScNahs9XMTk67jVYHipOTUYFiuLe1Z3EgHUT0




In [11]:
#all features and keys should be lowercase with underscore instead of spaces
def get_audiofeatures(url,track_uri,show=True):
    features = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness',
                'liveness','valence','tempo','time_signature']
    
    #dictionary 
    audiofeatures = {}
    
    #get url
    try:
        resp = requests.get(url=url,
                            headers={'Authorization': 'Bearer '+ token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
    #response in json format
    response = resp.json()
    
    audiofeatures.update({'track_uri':track_uri})
    

    for track_feature in features:
        feature_value = response[track_feature]
        audiofeatures.update({track_feature:feature_value}) #ties track_uri and features
    
    if show:
        print('URI: {uri} - AudioFeatures: {af}'.format(uri=track_uri,af=audiofeatures))
    
    return audiofeatures          

In [18]:
i = 0
master_track_audiofeatures=[]

start = time.time()

#Open file
with open('Data/artist_album_track_uri.csv') as csvfile:
    file = csv.reader(csvfile, delimiter='|')
    
    next(file) # Skip header
    head = [next(file) for x in range(4)] #for limiting amount of rows read
    
    for row in head:
        line = row[0].split('|') # list of items
        track_uri = row[3] #getting 1 item of the list
        print(i,line) 
        
        track_audiofeatures = get_audiofeatures('https://api.spotify.com/v1/audio-features/{id}'.format(id=track_uri),track_uri)
                                                
        #Is a list of the track_audiofeatures dictionary for each track_uri in file
        master_track_audiofeatures.append(track_audiofeatures)
        i+=1
stop = time.time()
duration = (stop - start)/60

0 ['3bBWKHfpepPOychRNFzg4q']
URI: 7cVA2h2HRlE2mOxhXEmtro - AudioFeatures: {'track_uri': '7cVA2h2HRlE2mOxhXEmtro', 'danceability': 0.503, 'energy': 0.656, 'key': 2, 'loudness': -4.768, 'mode': 1, 'speechiness': 0.0315, 'acousticness': 0.236, 'instrumentalness': 0.836, 'liveness': 0.355, 'valence': 0.472, 'tempo': 130.073, 'time_signature': 4}
1 ['3bBWKHfpepPOychRNFzg4q']
URI: 5N6IdYnIYaATgQKHg9CZJP - AudioFeatures: {'track_uri': '5N6IdYnIYaATgQKHg9CZJP', 'danceability': 0.258, 'energy': 0.54, 'key': 0, 'loudness': -6.73, 'mode': 1, 'speechiness': 0.0288, 'acousticness': 0.22, 'instrumentalness': 0.789, 'liveness': 0.0754, 'valence': 0.436, 'tempo': 83.048, 'time_signature': 4}
2 ['3bBWKHfpepPOychRNFzg4q']
URI: 2Fz3MpPkiUPEH6d4Heuipp - AudioFeatures: {'track_uri': '2Fz3MpPkiUPEH6d4Heuipp', 'danceability': 0.595, 'energy': 0.634, 'key': 7, 'loudness': -7.057, 'mode': 1, 'speechiness': 0.0303, 'acousticness': 0.783, 'instrumentalness': 0.747, 'liveness': 0.0989, 'valence': 0.489, 'tempo': 

In [19]:
master_track_audiofeatures

[{'track_uri': '7cVA2h2HRlE2mOxhXEmtro',
  'danceability': 0.503,
  'energy': 0.656,
  'key': 2,
  'loudness': -4.768,
  'mode': 1,
  'speechiness': 0.0315,
  'acousticness': 0.236,
  'instrumentalness': 0.836,
  'liveness': 0.355,
  'valence': 0.472,
  'tempo': 130.073,
  'time_signature': 4},
 {'track_uri': '5N6IdYnIYaATgQKHg9CZJP',
  'danceability': 0.258,
  'energy': 0.54,
  'key': 0,
  'loudness': -6.73,
  'mode': 1,
  'speechiness': 0.0288,
  'acousticness': 0.22,
  'instrumentalness': 0.789,
  'liveness': 0.0754,
  'valence': 0.436,
  'tempo': 83.048,
  'time_signature': 4},
 {'track_uri': '2Fz3MpPkiUPEH6d4Heuipp',
  'danceability': 0.595,
  'energy': 0.634,
  'key': 7,
  'loudness': -7.057,
  'mode': 1,
  'speechiness': 0.0303,
  'acousticness': 0.783,
  'instrumentalness': 0.747,
  'liveness': 0.0989,
  'valence': 0.489,
  'tempo': 108.009,
  'time_signature': 4},
 {'track_uri': '4yDo6bylEHQMbAaXLKSvwl',
  'danceability': 0.334,
  'energy': 0.744,
  'key': 2,
  'loudness': -5.

In [20]:
#Duration of script
str(duration)+' minutes'

'0.006715035438537598 minutes'

In [21]:
df = pd.DataFrame(master_track_audiofeatures)
df
#no_duplicates = df.drop_duplicates(['Album URI']) *Change to find duplicates of track_uri
#no_duplicates
#no_duplicates.to_csv('Data/artist_album_uri.csv',sep='|',index=False)

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,7cVA2h2HRlE2mOxhXEmtro,0.503,0.656,2,-4.768,1,0.0315,0.236,0.836,0.3550,0.472,130.073,4
1,5N6IdYnIYaATgQKHg9CZJP,0.258,0.540,0,-6.730,1,0.0288,0.220,0.789,0.0754,0.436,83.048,4
2,2Fz3MpPkiUPEH6d4Heuipp,0.595,0.634,7,-7.057,1,0.0303,0.783,0.747,0.0989,0.489,108.009,4
3,4yDo6bylEHQMbAaXLKSvwl,0.334,0.744,2,-5.317,1,0.0370,0.576,0.771,0.0860,0.639,169.865,3


In [ ]:
#This should be the output

#df = pd.DataFrame(master_track_audiofeatures)
#df
#no_duplicates = df.drop_duplicates(['Album URI']) *Change to find duplicates of track_uri
#no_duplicates
#no_duplicates.to_csv('Data/artist_album_uri.csv',sep='|',index=False)